# Ejercicio: análisis semántico con word2vec

<img src="img/word2vec.png" style="width:600px;height:400px;">

En este ejercicio vamos a utilizar word2vec para estudiar algunas relaciones semánticas entre palabras. Veremos cómo con esta técnica podemos resolver fácilmente los típicos problemas de encontrar palabras extrañas en un conjunto dado.

## Instrucciones

A lo largo de este cuaderno encontrarás celdas vacías que tendrás que rellenar con tu propio código. Sigue las instrucciones del cuaderno y presta especial atención a los siguientes iconos:

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Deberás responder a la pregunta indicada con el código o contestación que escribas en la celda inferior.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Esto es una pista u observación que te puede ayudar a resolver la práctica.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">Este es un ejercicio avanzado y voluntario que puedes realizar si quieres profundar más sobre el tema. Te animamos a intentarlo para aprender más ¡Ánimo!</td></tr>
</table>

Para evitar problemas de compatibilidad y de paquetes no instalados, se recomienda ejecutar este notebook bajo uno de los [entornos recomendados de Text Mining](https://github.com/albarji/teaching-environments/tree/master/textmining).

Adicionalmente si necesitas consultar la ayuda de cualquier función python puedes colocar el cursor de escritura sobre el nombre de la misma y pulsar Mayúsculas+Shift para que aparezca un recuadro con sus detalles. Ten en cuenta que esto únicamente funciona en las celdas de código.

¡Adelante!

## Carga y preparación de datos

Entrenar un modelo de word2vec es una tarea muy costosa computacionalmente, que además requiere de corpus de texto muy grandes, del orden de miles de millones de palabras. Afortunadamente existen modelos word2vec pre-entrenados que están disponibles de forma pública y con los que podemos trabajar para hacer nuestros análisis semánticos. Uno de ellos es el modelo [GoogleNews-vectors-negative300](https://code.google.com/archive/p/word2vec/), entrenado con 100.000 millones de palabras y que se encuentra disponible para descarga en la siguiente dirección: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Descarga manualmente el modelo <b>GoogleNews-vectors-negative300</b> en tu máquina, y a continuación crea una variable <i>modelfile</i> con la ruta del fichero descomprido del modelo.
  </td>
 </tr> 
</table>

In [1]:
####### INSERT YOUR CODE HERE
modelfile = "D:/Tmp/GoogleNews-vectors-negative300.bin"

## Similitud semántica

Un modelo pre-entrenado word2vec no es más que un diccionario en el que para cada palabra tenemos el vector que la representa. Podemos cargar en memoria este diccionario utilizando el paquete de análisis de texto **gensim**:

In [2]:
import gensim
embeddings = gensim.models.KeyedVectors.load_word2vec_format(modelfile, binary=True)

D:\Anaconda3\envs\textmining-labs\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Es conveniente aplicar la siguiente operación para guardar en memoria únicamente la información relativa a los vectores. Esto hace que no podamos reentrenar la representación vectorial de las palabras, pero para el objetivo de esta práctica no es necesario tal cosa.

In [3]:
embeddings.init_sims(replace=True)

Podemos comprobar ahora la representación vectorial de diferentes palabras, por ejemplo:

In [4]:
embeddings["queen"]

array([ 0.00173332, -0.04740431, -0.02289596,  0.0407935 ,  0.04353457,
       -0.02934553, -0.02354092, -0.07159019, -0.06514061,  0.01838126,
       -0.02499207, -0.12576653,  0.03434394, -0.00026957,  0.04385705,
        0.03724624,  0.02402463, -0.01547896,  0.02176728,  0.03111915,
        0.06288327,  0.04514696, -0.07803975, -0.03918111,  0.02160605,
       -0.01757507, -0.10190314,  0.03031296,  0.06223831, -0.05514379,
       -0.05159653, -0.04321209, -0.02724942,  0.07030027, -0.1173821 ,
       -0.04353457,  0.03176411,  0.08706914, -0.03128039,  0.06062592,
        0.03531137, -0.13737576,  0.08900401, -0.00915032,  0.0580461 ,
       -0.03724624, -0.00136046,  0.04804927,  0.05159653,  0.08835905,
       -0.00592554,  0.03257031,  0.01749445, -0.01031931, -0.05385388,
       -0.01918746, -0.11351236, -0.05707866,  0.03772996, -0.02982924,
        0.044502  ,  0.09222879, -0.01644639,  0.04288961,  0.0580461 ,
       -0.07320257, -0.0039302 ,  0.04643688, -0.05998096,  0.03

Aunque la representación vectorial de una palabra resulta oscura de interpretar, se ha comprobado que sigue una lógica semántica y sintáctica. Esto permite hacer aritmética con estos vectores y obtener resultados que son coherentes con lo que cabría esperar. Por ejemplo, si denotamos como $v(word)$ la representación vectorial de una cierta palabra *word* podemos encontrar casos como: 

$$v(king) - v(man) + v(woman) \simeq v(queen)$$

Podemos comprobar que eso es cierto utilizando la función **most_similar** del objeto que contiene los embeddings. Esta función recibe dos listas de palabras, a contribuir de forma positiva o negativa a la operación aritmética, y devuelve las palabras cuya representación vectorial sea más cercana al vector resultado de la operación, ordenadas por similitud:

In [5]:
embeddings.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431607246399),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.5181134343147278),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

¡Funciona! Podemos ver más ejemplos, como los siguientes:

In [6]:
embeddings.most_similar(positive=['Bush', 'Russia'], negative=['USA'])

[('Putin', 0.6845240592956543),
 ('President_Vladimir_Putin', 0.6755064725875854),
 ('Kremlin', 0.6264293193817139),
 ('Medvedev', 0.6225455403327942),
 ('Vladimir_Putin', 0.6010303497314453),
 ('President_George_W.', 0.5910208225250244),
 ('Prime_Minister_Vladimir_Putin', 0.5779234170913696),
 ('President_Dmitry_Medvedev', 0.5603682994842529),
 ('Medevedev', 0.545877993106842),
 ('George_W', 0.5436867475509644)]

In [7]:
embeddings.most_similar(positive=['Madrid', 'France'], negative=['Spain'])

[('Paris', 0.7502285242080688),
 ('Marseille', 0.603843092918396),
 ('French', 0.601784348487854),
 ('Colombes', 0.5965863466262817),
 ('Hopital_Europeen_Georges_Pompidou', 0.5867530107498169),
 ('Toulouse', 0.577813446521759),
 ('Parisian', 0.570327639579773),
 ('Cergy_Pontoise', 0.568040132522583),
 ('Marseilles', 0.5587785243988037),
 ('Strasbourg', 0.5559653043746948)]

In [8]:
embeddings.most_similar(positive=['goodbye', 'spanish'], negative=['english'])

[('adios', 0.6326478719711304),
 ('goodbyes', 0.6136816740036011),
 ('farewells', 0.5774319767951965),
 ('farewell', 0.5727684497833252),
 ('fond_farewell', 0.5445314645767212),
 ('hasta_luego', 0.5245710015296936),
 ('bid_farewell', 0.4987291693687439),
 ('hello', 0.488644003868103),
 ('arrivederci', 0.4783087372779846),
 ('adiós', 0.4729732871055603)]

In [ ]:
embeddings.most_similar(positive=['boat', 'air'], negative=['ocean'])

In [9]:
embeddings.most_similar(positive=['paella', 'Italy'], negative=['Spain'])

[('risotto', 0.6650041341781616),
 ('pasta', 0.6559766530990601),
 ('gnocchi', 0.6452398300170898),
 ('pizza_margherita', 0.6371422410011292),
 ('manicotti', 0.6349635720252991),
 ('osso_buco', 0.6342277526855469),
 ('di_pesce', 0.6324365735054016),
 ('ragu', 0.6315066814422607),
 ('pesce', 0.6304484605789185),
 ('minestrone', 0.6299175024032593)]

In [ ]:
embeddings.most_similar(positive=['Harry_Potter', 'evil'], negative=['good'])

In [10]:
embeddings.most_similar(positive=['artificial_intelligence', 'evil'], negative=['good'])

[('enslave_mankind', 0.5105279684066772),
 ('Necromongers', 0.5103692412376404),
 ('Artificial_Intelligence', 0.5079575181007385),
 ('malevolent', 0.5064362287521362),
 ('demon_lord', 0.5063003897666931),
 ('necromancer', 0.504085898399353),
 ('psionic_powers', 0.503738284111023),
 ('cybernetic_organisms', 0.4959980845451355),
 ('alien_beings', 0.49571239948272705),
 ('Overmind', 0.49549269676208496)]

Incluso pueden capturarse relaciones de morfología entre palabras, como las siguientes:

In [11]:
embeddings.most_similar(positive=['sister', 'he'], negative=['she'])

[('brother', 0.7627110481262207),
 ('younger_brother', 0.6856131553649902),
 ('cousin', 0.6685014963150024),
 ('uncle', 0.6580697298049927),
 ('nephew', 0.6526023149490356),
 ('father', 0.6411104202270508),
 ('son', 0.6308268308639526),
 ('elder_brother', 0.5854185819625854),
 ('brothers', 0.5706700086593628),
 ('twin_brother', 0.5622221827507019)]

In [12]:
embeddings.most_similar(positive=['cat', 'many'], negative=['one'])

[('cats', 0.5713158845901489),
 ('pets', 0.5167832970619202),
 ('kitties', 0.49727511405944824),
 ('pet', 0.484472393989563),
 ('feline', 0.48316147923469543),
 ('stray_cat', 0.4815112054347992),
 ('felines', 0.48110759258270264),
 ('kitten', 0.47646141052246094),
 ('dog', 0.47168344259262085),
 ('puppy', 0.46635887026786804)]

In [13]:
embeddings.most_similar(positive=['eat', 'past'], negative=['present'])

[('ate', 0.47409069538116455),
 ('eating', 0.45381370186805725),
 ('eaten', 0.447462260723114),
 ('eats', 0.43551111221313477),
 ('microwave_burrito', 0.40894240140914917),
 ('eat_buffets', 0.40705668926239014),
 ('banana_muffin', 0.39758825302124023),
 ('yo_yo_dieted', 0.3958820700645447),
 ('Rancho_Bernardo_Escondido', 0.3941839337348938),
 ('Filet_o_Fish', 0.3925513029098511)]

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    ¡Busca tú mismo algún ejemplo interesante!
  </td>
 </tr> 
</table>

In [17]:
####### INSERT YOUR CODE HERE
print(embeddings.most_similar(positive=[':)', 'sad'], negative=['happy']))

[('lol', 0.5809476375579834), ('.....', 0.5671467781066895), (':-(', 0.5583669543266296), ('haha', 0.5576198101043701), ('taylor_swift', 0.5430734157562256), ('hahaha', 0.5416510105133057), ('RT_@_@', 0.5390990972518921), ('hahah', 0.5386964678764343), ('HAHAHAHAHA', 0.5321018695831299), ('@_ChrisHarrisNFL', 0.5315563678741455)]


## Odd-one out

Una utilidad interesante de las distancias semánticas entre palabras es de resolver los típicos pasatiempos en los que se debe identificar la palabra que no encaja dentro de un grupo dado. Por ejemplo:

In [18]:
group = ["Obama", "Merkel", "Putin", "truck"]

Está claro que *truck* es la palabra intrusa en esta lista de presidentes del gobierno. Pero esto es algo que sabemos por nuestro amplio conocimiento del mundo y del lenguaje, y para un programa informático no es nada trivial llegar a esta conclusión. Sin embargo gracias a las representaciones semánticas en forma de vector que nos da word2vec podemos hacerlo.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Implementa una función <b>oddoneout</b> que recibe una lista de palabras y un modelo word2vec, y realice los siguientes pasos:
    <ol>
     <li> Obtener la representación vectorial de cada palabra recibida. Ignora las palabras para las que el modelo no contemple una representación vectorial.</li>
     <li> Calcula el vector medio de todas las palabras recibidas.</li>
     <li> Calcula la distancia de ese vector medio al vector representativo de cada palabra.</li>
     <li> Devuelve la palabra con mayor distancia.</li>
    </ol>
  </td>
 </tr> 
</table>

In [19]:
####### INSERT YOUR CODE HERE
import numpy as np
def oddoneout(words, embeddings):
    vectors = {word : embeddings[word] for word in words if word in embeddings}
    mean = np.mean([vectors[word] for word in vectors], axis=0)
    dists = {word : np.linalg.norm(vectors[word] - mean) for word in vectors}
    sorteddists = sorted(dists, key = lambda x : dists[x])
    return sorteddists[-1]

Vamos a comprobar ahora con el ejemplo de antes si la implementación ha funcionado:

In [20]:
oddoneout(group, embeddings)

'truck'

<table>
 <tr>
  <td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
    ¿Ha funcionado el ejemplo anterior? ¿Puedes pensar en otros ejemplos en los que el algoritmo también fucione? ¿Hay algún caso en el que falle?
     En gensim cualquier objeto de modelo word2vec dispone de la función <b>doesnt_match</b> que realiza la misma función que el algoritmo que has implementado, pero con un cálculo de distancias más adecuado a la representación vectorial. En general esta aproximación debería ser mejor que la que has implementado. ¿Encuentras algún caso en el que sea así?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE